<a href="https://colab.research.google.com/github/Ujjwal29/sentiment-Analysis/blob/master/CountVectorizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import nltk
import re
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

import warnings
warnings.filterwarnings("ignore")

In [0]:
df = pd.read_csv("train.csv")

In [4]:
df.head()

,id,original_text,lang,retweet_count,original_author,sentiment_class
0,1.245025e+18,Happy #MothersDay to all you amazing mothers o...,en,0,BeenXXPired,0
1,1.245759e+18,Happy Mothers Day Mum - I'm sorry I can't be t...,en,1,FestiveFeeling,0
2,1.246087e+18,Happy mothers day To all This doing a mothers ...,en,0,KrisAllenSak,-1
3,1.244803e+18,Happy mothers day to this beautiful woman...ro...,en,0,Queenuchee,0
4,1.244876e+18,Remembering the 3 most amazing ladies who made...,en,0,brittan17446794,-1


In [0]:
df.drop(columns=['id','original_author', 'lang', 'retweet_count'], inplace=True)

In [6]:
df.head()

,original_text,sentiment_class
0,Happy #MothersDay to all you amazing mothers o...,0
1,Happy Mothers Day Mum - I'm sorry I can't be t...,0
2,Happy mothers day To all This doing a mothers ...,-1
3,Happy mothers day to this beautiful woman...ro...,0
4,Remembering the 3 most amazing ladies who made...,-1


In [7]:
df.shape

(3235, 2)

In [8]:
df.sentiment_class.isnull().any()

False

In [9]:
df.original_text.isnull().any()

False

In [0]:
df = df.loc[df['original_text'].notna(), :]

In [0]:
text = df['original_text']
labels = df['sentiment_class']

In [12]:
#tokenize
text = text.apply(nltk.word_tokenize)
print('Tokenized')

#Remove stop words
stop_words = set(nltk.corpus.stopwords.words('english'))
text = text.apply(lambda x: [item for item in x if item not in stop_words])
print('Stop words Removed')


#Remove Punctuation
regex = '[a-z]+'
text = text.apply(lambda x: [item for item in x if re.match(regex, item)])
print('Punctuation Removed')

#Lemmetization
lem = nltk.stem.wordnet.WordNetLemmatizer()
text = text.apply(lambda x: [lem.lemmatize(item, pos='v') for item in x])
print('Lemmetize')

Tokenized
Stop words Removed
Punctuation Removed
Lemmetize


In [0]:
# text

In [0]:
text = [" ".join(sen) for sen in text.values]

In [0]:
def vectorize(vec, x_train, x_test):
    x_train_vec = vec.fit_transform(x_train)
    x_test_vec = vec.transform(x_test)
    
    print("Vectorization Complete!")
    
    return x_train_vec, x_test_vec

In [16]:
from sklearn.feature_extraction.text import CountVectorizer


X_train, X_test, y_train, y_test = train_test_split(text, labels, test_size=0.2, shuffle=True)

X_train_vec, X_test_vec = vectorize(CountVectorizer(), X_train, X_test)

Vectorization Complete!


## Gradient Boosting Classifier

In [0]:
#Building the model
gbc = GradientBoostingClassifier()

# param = { 
#     'learning_rate': [0.05, 0.1, 0.01], 
#     'min_samples_split': [2, 5, 7, 10],
#     'n_estimators':[50, 100, 150, 200, 400, 500],
#     'max_features':[5,10,15,20,25],
#     'max_depth':[3,5,10,20,25,30]}

In [18]:
gbc = GradientBoostingClassifier(learning_rate=0.05, 
                                 max_depth=3, 
                                 max_features=10, 
                                 min_samples_split=7, 
                                 n_estimators=500)
gbc.fit(X_train_vec, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.05, loss='deviance', max_depth=3,
                           max_features=10, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=7,
                           min_weight_fraction_leaf=0.0, n_estimators=500,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [19]:
#Predictions
from sklearn.model_selection import cross_val_score

y_pred = gbc.predict(X_test_vec)

print("Precision: %1.3f " % (precision_score(y_test, y_pred, average='macro')))
print("Recall: %1.3f " %  recall_score(y_test, y_pred, average='macro'))
print("F1: %1.3f" % f1_score(y_test, y_pred, average='macro'))
scores = cross_val_score(gbc, X_test_vec, y_test, cv=5)
print("Score Mean: %1.3f" % (scores.mean()))
print("GBoosting Score Accuracy", str(round(accuracy_score(y_pred,y_test)*100, 2)) + "%")

Precision: 0.165 
Recall: 0.333 
F1: 0.221
Score Mean: 0.481
GBoosting Score Accuracy 49.61%


## LightGBM

In [0]:
import lightgbm

In [0]:
# lgbm = lightgbm.LGBMClassifier(num_leaves=80,
#                               max_depth=-1,
#                               learning_rate = 0.05,
#                               n_estimators=8,
#                               n_jobs=-1,
#                               reg_alpha=0.0)

# lgbm.fit(X_train_vec, y_train)

# y_pred = lgbm.predict(X_test_vec)

# print("LGBM Score Accuracy", str(round(accuracy_score(y_pred,y_test)*100, 2)) + "%")

## Notes on Gradient Boosting

1. Using a small max_features value can significantly decrease the runtime.


2. The core principle of AdaBoost is to fit a sequence of weak learners 
(i.e., models that are only slightlybetter than random guessing, such as small decision trees) on repeatedly modified versions of the data.


3. The predictions from all of them are then combined through a weighted 
majority vote (or sum) to produce the final prediction.


4. GradientBoostingClassifier supports both binary and multi-class classification


5. The number of weak learners (i.e. regression trees) is controlled by the parameter n_estimators; The size of each tree can be controlled either by setting the tree depth via max_depth or by setting the number of leaf nodes via max_leaf_nodes. The learning_rate is a hyper-parameter in the range (0.0, 1.0] that controls overfitting via shrinkage .


6. Both GradientBoostingRegressor and GradientBoostingClassifier support warm_start=True which allows you to add more estimators to an already fitted model.


7. Scikit-learn 0.21 introduced two new experimental implementations of gradient boosting trees, namely HistGradientBoostingClassifier and HistGradientBoostingRegressor, inspired by LightGBM (See [LightGBM]).


8. These histogram-based estimators can be orders of magnitude faster than GradientBoostingClassifier and GradientBoostingRegressor when the number of samples is larger than tens of thousands of samples.


9. They also have built-in support for missing values, which avoids the need for an imputer.

In [22]:
'''
check for the following algos

1. LightGBM
2. XGBoost
3. HistGradientBoostingClassifier
4. 
'''

'\ncheck for the following algos\n\n1. LightGBM\n2. XGBoost\n3. HistGradientBoostingClassifier\n4. \n'